In [1]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. CSV 파일 로드
input_file = "C:/Users/hp/Desktop/Bootcamp/PROJECT_OTT_AARRR/preprocessed_data_unique.csv"
df = pd.read_csv(input_file)

# 2. categoryID가 6인 데이터만 필터링
df_filtered = df[df['categoryID'] == 6].reset_index(drop=True)
print("Filtering: {} rows found with categoryID==6".format(df_filtered.shape[0]))

# 3. combined_text 열 생성 (만약 없는 경우, title, tags, img_text를 결합)
if "combined_text" not in df_filtered.columns:
    df_filtered["combined_text"] = (
        df_filtered["title"].fillna("") + " " +
        df_filtered["tags"].fillna("") + " " +
        df_filtered["img_text"].fillna("")
    )

# 4. 텍스트 데이터를 문서 리스트로 생성
docs = df_filtered["combined_text"].fillna("").tolist()

# 5. TF-IDF 행렬 생성
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(docs)

# 6. NMF 모델 학습 (토픽 개수를 10으로 설정)
n_topics = 10
nmf_model = NMF(n_components=n_topics, random_state=42)
W = nmf_model.fit_transform(tfidf)  # 문서-토픽 행렬

# 7. 각 문서에 대해 가장 큰 기여를 보인 토픽을 예측(문서별 argmax)
df_filtered["predicted_topic"] = W.argmax(axis=1)

# 8. 토픽 번호에 따른 새로운 categoryID 매핑 딕셔너리 정의
topic_to_category = {
    0: 1,
    6: 1,
    7: 1,
    2: 3
}

# 매핑되지 않은 토픽(예: 1, 3, 4, 5, 8, 9)은 기본값 6로 처리
df_filtered["new_categoryID"] = df_filtered["predicted_topic"].map(topic_to_category)
df_filtered["new_categoryID"] = df_filtered["new_categoryID"].fillna(6)

# 9. 원본 DataFrame에서 categoryID가 6인 행에 대해 재매핑된 categoryID로 업데이트
# (여기서는 videoID가 고유하다고 가정하고, 필터링 순서가 동일하므로 index 기반으로 대체)
df_updated = df.copy()
df_updated.loc[df_updated['categoryID'] == 6, 'categoryID'] = df_filtered["new_categoryID"].values

# 10. 결과를 새로운 CSV 파일로 저장
output_file = "preprocessed_data_unique_updated.csv"
df_updated.to_csv(output_file, index=False)

print("categoryID 재매핑이 완료되었습니다. 결과 파일:", output_file)


Filtering: 57039 rows found with categoryID==6
categoryID 재매핑이 완료되었습니다. 결과 파일: preprocessed_data_unique_updated.csv
